In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Portfolio Manager: Enables adding / removal positions and monthly / quarterly rebalances. Also allows testing by using mock returns to show how rebalancing changes performance.

| Method                  | Purpose                                   |
| ----------------------- | ----------------------------------------- |
| `__init__()`            | Initialize portfolio, capital, and config |
| `add_position()`        | Buy shares                                |
| `remove_position()`     | Sell shares                               |
| `get_portfolio_value()` | Compute current total portfolio value     |
| `rebalance()`           | Reallocate portfolio based on ranked list |
| `simulate_returns()`    | Simulate time-series performance          |
| `plot_performance()`    | Visualize portfolio performance           |

* Core State: self.cash and self.positions (which holds current shares and the price they were last valued at, used as the entry/closing price for the next period's return).
* Time Step Logic (simulate_returns):
    1. Apply Returns: The backtester (or simulate_returns loop) is responsible for applying the period's return to the prior closing price (entry_price) to determine the new current_prices.
    2. Rebalance Check: It checks the date.month against the rebalance_freq to trigger the rebalance method.
    3. Update State: After returns and any rebalancing, the current price is saved back into the entry_price field of self.positions.
* Rebalancing Logic: The rebalance method uses the total portfolio value and target_weights to calculate necessary trades. It always sells first to liquidate over-allocated positions (increasing cash) and then buys second to re-invest in under-allocated positions (consuming cash).
* Audit Trail: The self.transactions list is crucial for tracking every trade (Buy/Sell, quantity, price) for downstream analysis of trading costs and turnover.

In [14]:
class PortfolioManager:
    def __init__(self, initial_capital: float, rebalance_freq: str = 'monthly'):
        """
        Initializes a portfolio manager.

        Parameters:
        - initial_capital: starting cash balance
        - rebalance_freq: 'monthly' or 'quarterly' updates
        """
        self.initial_capital = initial_capital
        self.cash = initial_capital
        self.positions = {}   # stores {ticker: {'shares': int, 'price': float}}
        self.history = []     # track portfolio value over time
        self.rebalance_freq = rebalance_freq.lower()
        self.transactions = [] # added for backtester audit trail

    def __repr__(self):
        mock_prices = {t: p['entry_price'] for t, p in self.positions.items()}
        return f"<PortfolioManager | Value: ${self.get_portfolio_value(mock_prices):,.2f} | Cash: ${self.cash:,.2f} | Positions: {len(self.positions)}>"

    def get_current_prices(self):
            """Helper function to get current prices from positions (for repr)."""
            return {t: p['entry_price'] for t, p in self.positions.items()}

    def add_position(self, ticker: str, shares: float, price: float, date='N/A'):
        """
        Add a new stock position to the portfolio, also allowing fractional shares.
        Deducts cost from available cash.
        """
        cost = shares * price
        if cost > self.cash:
            raise ValueError(f"Insufficient capital (${self.cash:,.2f}) to purchase {shares:.4f} of {ticker} (Cost: ${cost:,.2f})")

        # Update weighted average entry price
        if ticker in self.positions:
            current_qty = self.positions[ticker]['shares']
            current_cost = current_qty * self.positions[ticker]['entry_price']
            new_qty = current_qty + shares
            # Avoid division by zero if new_qty is zero (shouldn't happen here)
            new_entry_price = (current_cost + cost) / new_qty if new_qty > 0 else 0
            self.positions[ticker]['shares'] = new_qty
            self.positions[ticker]['entry_price'] = new_entry_price
        else:
            self.positions[ticker] = {'shares': shares, 'entry_price': price}

        self.cash -= cost
        self.transactions.append({'date': date, 'type': 'BUY', 'asset': ticker,
                                  'quantity': shares, 'price': price, 'cost': cost})

        print(f"Traded: BUY {shares:.4f} of {ticker} at ${price:.2f}")

    def remove_position(self, ticker: str, shares: float, price: float, date='N/A'):
        """
        Sell all shares of a stock at the current market price.
        Adds proceeds to cash.
        """
        if ticker not in self.positions:
            print(f"WARNING: {ticker} not present in current portfolio. Cannot sell.")
            return

        current_shares = self.positions[ticker]['shares']
        if shares > current_shares:
             raise ValueError(f"Cannot sell {shares:.4f} of {ticker}, only {current_shares:.4f} held.")

        proceeds = shares * price
        self.cash += proceeds
        self.positions[ticker]['shares'] -= shares

        # Remove position if shares are negligible (close to zero)
        if abs(self.positions[ticker]['shares']) < 1e-6:
            del self.positions[ticker]

        self.transactions.append({'date': date, 'type': 'SELL', 'asset': ticker,
                                  'quantity': shares, 'price': price, 'proceeds': proceeds})

        print(f"Traded: SELL {shares:.4f} of {ticker} at ${price:.2f}")

    def get_portfolio_value(self, current_prices: dict):
        """
        Calculate total portfolio value (cash + stock holdings).
        """
        stock_value = 0
        for ticker, info in self.positions.items():
            if ticker in current_prices:
                stock_value += info['shares'] * current_prices[ticker]
        total_value = self.cash + stock_value
        return total_value

    def rebalance(self, target_weights: dict, current_prices: dict, date='N/A'):
        """
        Rebalance the portfolio to match the target_weights.

        Logic: Prioritize selling over-allocated assets first to generate cash,
        then buy under-allocated assets.
        """
        total_value = self.get_portfolio_value(current_prices)

        # 1. Determine target value for each asset
        target_values = {ticker: total_value * weight for ticker, weight in target_weights.items()}

        # 2. Sell Phase (Sell Over-allocated Assets)
        assets_to_adjust = list(self.positions.keys()) # Copy keys to iterate safely
        for ticker in assets_to_adjust:
            if ticker not in current_prices: continue

            current_shares = self.positions.get(ticker, {}).get('shares', 0)
            current_value = current_shares * current_prices[ticker]
            target_value = target_values.get(ticker, 0) # Target might be 0 if not in target_weights

            if current_value > target_value:
                value_to_sell = current_value - target_value
                shares_to_sell = value_to_sell / current_prices[ticker]

                # Ensure we don't sell more than we own
                shares_to_sell = min(shares_to_sell, current_shares)

                if shares_to_sell > 1e-6:
                    self.remove_position(ticker, shares_to_sell, current_prices[ticker], date)

        # 3. Buy Phase (Buy Under-allocated Assets)
        for ticker, target_value in target_values.items():
            if ticker not in current_prices: continue

            current_shares = self.positions.get(ticker, {}).get('shares', 0)
            current_value = current_shares * current_prices[ticker]

            if current_value < target_value:
                value_to_buy = target_value - current_value

                # Check how much cash is available
                value_to_buy = min(value_to_buy, self.cash)

                shares_to_buy = value_to_buy / current_prices[ticker]

                if shares_to_buy > 1e-6:
                    self.add_position(ticker, shares_to_buy, current_prices[ticker], date)

        print(f"Rebalanced on {date}. New Cash: ${self.cash:,.2f}")

    def simulate_returns(self, returns_df: pd.DataFrame, initial_prices: dict, target_weights: dict):
            """
            Simulate portfolio value changes over time with periodic rebalancing.

            Parameters:
            - returns_df: DataFrame of monthly returns {Date: {Ticker: Return}}
            - initial_prices: Starting price for each ticker
            - target_weights: The desired allocation {Ticker: Weight}
            """
            self.history = []

            # 0. Initial Setup: Set entry price for the first period
            initial_cash_used = 0
            for ticker, price in initial_prices.items():
                if ticker in target_weights:
                    initial_value = self.initial_capital * target_weights[ticker]
                    shares = initial_value / price

                    # Use a specific entry_price for initialization
                    self.positions[ticker] = {'shares': shares, 'entry_price': price}
                    initial_cash_used += initial_value

            self.cash = self.initial_capital - initial_cash_used

            print(f"Initialized with {len(self.positions)} assets. Remaining cash: ${self.cash:,.2f}")

            # 1. Simulation Loop
            for date, returns in returns_df.iterrows():
                # --- A. Apply Returns and Determine Current Prices ---
                current_prices = {}
                for ticker, position in self.positions.items():
                    # Get return for the period, default to 0 if asset isn't in returns_df for this date
                    ret = returns.get(ticker, 0)

                    # Calculate the current price by applying the return to the prior closing price (entry_price)
                    current_price = position['entry_price'] * (1 + ret)
                    current_prices[ticker] = current_price

                # --- B. Rebalance Check ---
                is_rebalance_month = self.rebalance_freq == 'monthly' or self.rebalance_freq == 'm'
                # Check for quarter-end months (Mar, Jun, Sep, Dec)
                is_rebalance_quarter = (self.rebalance_freq == 'quarterly' or self.rebalance_freq == 'q') and date.month in [3, 6, 9, 12]

                if is_rebalance_month or is_rebalance_quarter:
                    # Rebalance requires target_weights and current prices
                    self.rebalance(target_weights, current_prices, date=date)

                # --- C. Track Value and Update Prices for Next Period ---

                # Calculate the final value after returns and any rebalancing trades
                value = self.get_portfolio_value(current_prices)
                self.history.append({'date': date, 'value': value})

                # Update the 'entry_price' to the current closing price for the next period's return calculation.
                for ticker, price in current_prices.items():
                    if ticker in self.positions:
                        self.positions[ticker]['entry_price'] = price

            return pd.DataFrame(self.history)

    def plot_performance(self):
        df = pd.DataFrame(self.history)
        if df.empty:
            print("No performance data to plot.")
            return

        # Calculate Cumulative Return
        initial_value = df['value'].iloc[0]
        final_value = df['value'].iloc[-1]
        cagr = ((final_value / initial_value)**(12/len(df)) - 1) * 100

        plt.figure(figsize=(12, 6))
        plt.plot(df['date'], df['value'], label='Portfolio Value')
        plt.title(f'Portfolio Performance ({self.rebalance_freq.capitalize()} Rebalance) | CAGR: {cagr:.2f}%', fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value ($)')
        plt.grid(True, axis='y', linestyle='--')
        plt.legend()
        plt.show()

Generating Mock Returns

In [15]:
def generate_mock_returns(assets: list, periods: int, avg_return: float = 0.005, std_dev: float = 0.02):
    """Generates a DataFrame of mock monthly returns."""
    np.random.seed(42) # For reproducibility
    data = np.random.normal(avg_return, std_dev, size=(periods, len(assets)))
    dates = pd.date_range(start='2020-01-01', periods=periods, freq='ME')
    
    returns_df = pd.DataFrame(data, index=dates, columns=assets)
    
    # Introduce a trend for visual difference (e.g., Asset 1 strongly outperforms)
    returns_df.iloc[:, 0] += np.linspace(0.01, 0.001, periods)
    
    return returns_df

Testing Setup

In [16]:
## Simulation Setup
INITIAL_CAPITAL = 100000
ASSETS = ['STOCK_A', 'STOCK_B', 'STOCK_C', 'STOCK_D']
INITIAL_PRICES = {'STOCK_A': 100, 'STOCK_B': 50, 'STOCK_C': 200, 'STOCK_D': 75}
TARGET_WEIGHTS = {asset: 1/len(ASSETS) for asset in ASSETS} # Equal Weight
SIMULATION_PERIODS = 48 # 4 years of data

# Generate Mock Returns Data
mock_returns = generate_mock_returns(ASSETS, SIMULATION_PERIODS)

## Test Case 1: Monthly Rebalancing
print("--- Running Monthly Rebalance Simulation ---")
manager_monthly = PortfolioManager(INITIAL_CAPITAL, rebalance_freq='monthly')
history_monthly = manager_monthly.simulate_returns(mock_returns, INITIAL_PRICES, TARGET_WEIGHTS)
manager_monthly.plot_performance()

## Test Case 2: Quarterly Rebalancing
print("\n--- Running Quarterly Rebalance Simulation ---")
manager_quarterly = PortfolioManager(INITIAL_CAPITAL, rebalance_freq='quarterly')
history_quarterly = manager_quarterly.simulate_returns(mock_returns, INITIAL_PRICES, TARGET_WEIGHTS)
manager_quarterly.plot_performance()

## Performance Comparison
plt.figure(figsize=(12, 6))
plt.plot(history_monthly['date'], history_monthly['value'], label='Monthly Rebalance', linewidth=2)
plt.plot(history_quarterly['date'], history_quarterly['value'], label='Quarterly Rebalance', linestyle='--', alpha=0.7)
plt.title('Performance Comparison: Monthly vs. Quarterly Rebalancing', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.grid(True, axis='y', linestyle='--')
plt.legend()
plt.show()

print("\n--- Final Results ---")
print(f"Monthly Rebalance Final Value: ${history_monthly['value'].iloc[-1]:,.2f}")
print(f"Quarterly Rebalance Final Value: ${history_quarterly['value'].iloc[-1]:,.2f}")

--- Running Monthly Rebalance Simulation ---
Initialized with 4 assets. Remaining cash: $0.00
Traded: SELL 1.1680 of STOCK_A at $102.49
Traded: SELL 4.9301 of STOCK_D at $77.66
Traded: BUY 8.9356 of STOCK_B at $50.11
Traded: BUY 0.2692 of STOCK_C at $203.59
Rebalanced on 2020-01-31 00:00:00. New Cash: $0.00
Traded: SELL 2.3856 of STOCK_C at $211.04
Traded: SELL 1.1280 of STOCK_D at $79.24
Traded: BUY 1.6550 of STOCK_A at $103.53


ValueError: Insufficient capital ($421.49) to purchase 8.4083 of STOCK_B (Cost: $421.49)

Notes
- rebalance() can be called each month or quarter.
- Connect with ranking system output (ranked_stocks).
- Use simulate_returns() to feed value history into backtester.
- Cash balance automatically updates on buys/sells.
- Portfolio value can be logged at each rebalance step.